IMPORTS

In [166]:
import pandas as pd
import numpy as np

# <b>Importing Data <b>

In [167]:
df_branch_service = pd.read_json("branch_service_transaction_info.json")

## DATA PROFILING

<b>INITIAL SHAPE ANG UNIQUE ROWS<b>

In [168]:
print(df_branch_service.shape)
print(df_branch_service['txn_id'].nunique())


(130653, 4)
62354


In [169]:
df_branch_service['price'].describe()

count    110653.000000
mean         95.966969
std         115.874117
min           0.000000
25%          30.123790
50%          66.123457
75%          88.093930
max         400.231230
Name: price, dtype: float64

In [170]:
print(df_branch_service['branch_name'].unique())   
print(df_branch_service['service'].unique()) 
print(df_branch_service['price'].unique()) 
  

['MallOfAsia' 'Starmall' 'SmallMall' 'MayMall' 'FrankMall' 'Megamall'
 'RobinsonsMall' '' None 'N/A']
['Manicure' 'HairColor' 'FootSpa' 'Rebond' 'Haircut' 'NailColor'
 'Pedicure']
[         nan   0.          30.1237897   66.12345678  77.987989
 100.12123    400.23123     55.2324      88.09393   ]


### REMOVE DUPLICATES


In [171]:
print(df_branch_service.shape)
df_branch_service = df_branch_service.drop_duplicates()
print(df_branch_service.shape)



(130653, 4)
(118276, 4)


REMOVE ALMOST EXACT DUPLICATES

In [172]:
cols = df_branch_service.columns.tolist()
cols.remove('txn_id')

for col in cols:
    observed_cols = df_branch_service.drop(col, axis=1).columns.tolist()
    df_branch_service.drop_duplicates(observed_cols, keep='first', inplace=True)

print(df_branch_service.shape)
print(df_branch_service.nunique())

(62355, 4)
txn_id         62354
branch_name        9
service            7
price              8
dtype: int64


In [173]:
df_branch_service.to_parquet('branch_service_duplicates_removed.parquet')

REMOVE NULL VALUES

In [174]:
df_branch_service = df_branch_service.dropna(subset=['branch_name'])
df_branch_service = df_branch_service.dropna(subset=['service'])
df_branch_service = df_branch_service.dropna(subset=['price'])
print(df_branch_service.shape)



(43985, 4)


REMOVE MISSING VALUES

In [175]:
##df_branch_service = df_branch_service.drop(df_branch_service[df_branch_service['price'] == 0].index)
df_branch_service = df_branch_service.drop(df_branch_service[df_branch_service['branch_name'] == ''].index)
df_branch_service = df_branch_service.drop(df_branch_service[df_branch_service['branch_name'] == 'N/A'].index)



print(df_branch_service.shape)
df_branch_service.to_parquet('branch_service_nullbranch_removed.parquet')

(40962, 4)


### Formatting Values

CHANGE VALUES TO CAMELCASE

In [176]:
df_branch_service.loc[df_branch_service['branch_name'] == 'Starmall', 'branch_name'] = 'StarMall'
df_branch_service.loc[df_branch_service['branch_name'] == 'Megamall', 'branch_name'] = 'MegaMall'

ROUND  PRICES TO 2 DECIMAL PLACES

In [177]:
df_branch_service = df_branch_service.round({'price': 2})

In [178]:
df_branch_service.to_parquet('branch_service_formatted_values.parquet')

CHECKER

In [179]:
##df_branch_service.sort_values(by=['txn_id'], inplace=True, ascending=False)     
df_branch_service.head(30)

,txn_id,branch_name,service,price
20000,TXN-05451,MayMall,HairColor,0.0
20002,TXN-02255,FrankMall,Rebond,0.0
20006,TXN-59105,FrankMall,FootSpa,0.0
20008,TXN-57176,MallOfAsia,Rebond,0.0
20011,TXN-13049,SmallMall,Pedicure,0.0
20013,TXN-54729,MallOfAsia,Rebond,0.0
20015,TXN-53352,StarMall,Rebond,0.0
20017,TXN-38984,FrankMall,NailColor,0.0
20020,TXN-13722,MayMall,Manicure,0.0
20022,TXN-59402,MegaMall,Rebond,0.0
